In [ ]:
!rm -r /content/*

In [ ]:
!pip install flask_ngrok
!pip install -U flask-cors
!pip install mtcnn
!pip install pillow

     |████████████████████████████████| 2.3MB 8.1MB/s 


In [ ]:
import os
import io
import cv2
import shutil
import random
import sqlite3
import numpy as np
from PIL import Image
from mtcnn import MTCNN
from matplotlib import rcParams
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.preprocessing import image
from matplotlib.patches import Rectangle
from scipy.spatial.distance import cosine
from google.colab.patches import cv2_imshow
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/capstone/P1E_S1.zip /content
!cp /content/drive/MyDrive/capstone/P1E_S2.zip /content
!cp /content/drive/MyDrive/capstone/P1E_S3.zip /content
!cp /content/drive/MyDrive/capstone/VGG16.h5 /content
!cp /content/drive/MyDrive/capstone/labels.zip /content

In [ ]:
!rm -r /content/P1E_S1
!rm -r /content/P1E_S2
!rm -r /content/P1E_S3

!unzip /content/P1E_S1.zip
!unzip /content/P1E_S2.zip
!unzip /content/P1E_S3.zip
!unzip /content/labels.zip

Streaming output truncated to the last 5000 lines.
  inflating: P1E_S2/cam2/Person4/img177_163.jpg  
  inflating: P1E_S2/cam2/Person4/img178_163.jpg  
  inflating: P1E_S2/cam2/Person4/img179_163.jpg  
  inflating: P1E_S2/cam2/Person4/img180_163.jpg  
  inflating: P1E_S2/cam2/Person4/img181_163.jpg  
  inflating: P1E_S2/cam2/Person4/img182_163.jpg  
  inflating: P1E_S2/cam2/Person4/img183_163.jpg  
  inflating: P1E_S2/cam2/Person4/img184_163.jpg  
  inflating: P1E_S2/cam2/Person4/img185_163.jpg  
  inflating: P1E_S2/cam2/Person4/img186_163.jpg  
  inflating: P1E_S2/cam2/Person4/img187_163.jpg  
  inflating: P1E_S2/cam2/Person4/img188_163.jpg  
  inflating: P1E_S2/cam2/Person4/img189_163.jpg  
  inflating: P1E_S2/cam2/Person4/img190_163.jpg  
  inflating: P1E_S2/cam2/Person4/img191_163.jpg  
  inflating: P1E_S2/cam2/Person4/img192_163.jpg  
  inflating: P1E_S2/cam2/Person4/img193_163.jpg  
  inflating: P1E_S2/cam2/Person4/img194_163.jpg  
  inflating: P1E_S2/cam2/Person4/img195_163.jpg  

# **Classes**

In [ ]:
class ImagePreprocessing:

  def hist_equ(self,image):
    pixels = cv2.imread(image,0)
    x=cv2.equalizeHist(pixels)
    pixels = cv2.cvtColor(x,cv2.COLOR_GRAY2RGB)
    cv2.imwrite(image, pixels)

  def crop_image(self,image,result_list):
    pixels = cv2.imread(image)
    data = cv2.cvtColor(pixels, cv2.COLOR_BGR2RGB)
    
    padd = 10; max = 0; max_x=0; max_y=0; max_width=0; max_height=0

    for result in result_list:
      x, y, width, height = result['box']
      if (width*height>max):
        max_x, max_y, max_width, max_height = result['box']  
        max = width*height
    
    im = data[max_y-padd+1:max_y+max_height+padd, max_x-padd+1:max_x+max_width+padd]
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    cv2.imwrite(image, im)

  def image_resize(self,image):
    img = Image.open(image)
    newsize = (48, 48) 
    img = img.resize(newsize)
    img.save(image)

**Show the effect of histogram Equalization**


In [ ]:
preObj = ImagePreprocessing()
img = '/content/00000843.jpg'
img_A = mpimg.imread(img)

preObj.hist_equ(img)

%matplotlib inline

rcParams['figure.figsize'] = 11 ,8
img_B = mpimg.imread(img)

fig, ax = plt.subplots(1,2)
ax[0].imshow(img_A);
ax[1].imshow(img_B);

In [ ]:
class FaceDetection:

  def mtcnn_facedetect(self,image):
    pixels = cv2.imread(image,3)
    detector = MTCNN()
    res = detector.detect_faces(pixels)
    return res

**Show the detection of face by MTCNN**

In [ ]:
faceObj = FaceDetection()
img = '/content/00000843.jpg'
faces = faceObj.mtcnn_facedetect(img)


data = plt.imread(img)
plt.imshow(data)
ax = plt.gca()

for face in faces:

  x, y, width, height = face['box']
  rect = Rectangle((x, y), width, height, fill=False, color='red')
  ax.add_patch(rect)
plt.show()

**Show cropping of image**

In [ ]:
preObj.crop_image(img,faces)
im_data = cv2.imread(img)
print("Face shape:",im_data.shape)
cv2_imshow(im_data)

**Show resizing of image to (224,224)**

In [ ]:
preObj.image_resize(img)
im_data = cv2.imread(img)
print("Face shape:",im_data.shape)
cv2_imshow(im_data)

**Show the model**

In [ ]:
class Classification:

  def __init__(self,image):
    self.originalImage = image
    self.image = self.originalImage

    im = "/tmp/"+(self.image).split("/")[-1]
    shutil.copy2(self.image, im)
    self.image = im

    # Instantiate class objects
    self.preprocess_object = ImagePreprocessing()
    self.face_object = FaceDetection()
    # self.feature_object = FeatureExtraction()

  # Compare the i/p vector with whole data and return best match
  def find_match(self,inp_vector):
    min = 1.1
    match =''
    for data in train:
      val = cosine(inp_vector,data[0])
      if (val<min):
        min = val
        match = data[2]
    return match
  
  def classify(self):
    # Do histogram equalization
    # self.preprocess_object.hist_equ(self.image)

    # Detect face using MTCNN
    face_coordinates = self.face_object.mtcnn_facedetect(self.image)
    if len(face_coordinates)<1:
      return "0"
    # Crop and resize the image to (224,224)
    self.preprocess_object.crop_image(self.image, face_coordinates)
    # self.preprocess_object.image_resize(self.image)

    # Extract features using ResNet50
    # inp_vector = self.feature_object.feature_extract(self.image)
    # # Find the match in database
    # return self.find_match(inp_vector)
    return find_match(self.image)

In [ ]:
class User:

  def __init__(self):
    self.image = ''
    # self.match = ''
  
  def upload(self,image):
    self.image = image
  
  def display(self,match):
    %matplotlib inline

    rcParams['figure.figsize'] = 11 ,8
    print(self.image,match)
    img_A = mpimg.imread(self.image)
    img_B = mpimg.imread(match)

    fig, ax = plt.subplots(1,2)
    ax[0].imshow(img_A);
    ax[1].imshow(img_B);

  def get_results(self):
    obj = Classification(self.image)
    match = obj.classify()
    # self.display(match)
    return match

In [ ]:
import cv2
from keras.models import load_model
import numpy as np
input_shape = (48,48,3)

model = load_model('/content/drive/MyDrive/capstone/new_VGG16.h5')

def find_match(im_path):
  im = cv2.imread(im_path)
  im = cv2.resize(im,(48,48))
  im = np.array(im)
  im = np.resize(im,input_shape)/255

  temp = [im]
  temp = np.array(temp)

  # temp.shape
  # np.array()

  match = model.predict(temp)
  match = np.argmax(match)
  print(match+1)
  return match+1

In [ ]:
!unzip /content/static.zip
!unzip /content/templates.zip
!rm -r /content/static.zip
!rm -r /content/templates.zip
!mv /content/content/* /content/
!rm -r /content/content

Archive:  /content/static.zip
   creating: content/static/
  inflating: content/static/index.css  
  inflating: content/static/index.js  
  inflating: content/static/login.css  
   creating: content/static/.ipynb_checkpoints/
Archive:  /content/templates.zip
   creating: content/templates/
  inflating: content/templates/login.html  
  inflating: content/templates/add.html  
  inflating: content/templates/index.jinja2  
  inflating: content/templates/home.html  
  inflating: content/templates/signup.html  
  inflating: content/templates/admin.html  
  inflating: content/templates/index.html  
   creating: content/templates/.ipynb_checkpoints/


In [ ]:

import zipfile
def unzip_file(zip_src, dst_dir):

  r = zipfile.is_zipfile(zip_src)
  if r:
      fz = zipfile.ZipFile(zip_src, "r")
      for file in fz.namelist():
          fz.extract(file, dst_dir)
  else:
      return "Please upload zip file"

In [ ]:
# from keras.models import load_model
# model = load_model('/content/VGG16.h5')
# def add_class(): 
#  #-----------------
#   from keras.models import Model
#   import numpy as np

#   # save the original weights
#   weights_bak = model.layers[-1].get_weights()
#   nb_classes = model.layers[-1].output_shape[-1]
#   # print(len(weights_bak[1]))
#   model.layers.pop()
#   # model.summary()
#   # print("--------------------")
#   new_layer = Dense(nb_classes + 1, activation='softmax')
#   out = new_layer(model.layers[-2].output)
#   inp = model.input
#   model = Model(inp, out)
#   # model.summary()
#   weights_new = model.layers[-1].get_weights()

#   # copy the original weights back
#   weights_new[0][:, :-1] = weights_bak[0]
#   weights_new[1][:-1] = weights_bak[1]

#   # use the average weight to init the new class.
#   weights_new[0][:, -1] = np.mean(weights_bak[0], axis=1)
#   weights_new[1][-1] = np.mean(weights_bak[1])

#   model.layers[-1].set_weights(weights_new)

#   #-----------------------
#   from keras.optimizers import Adam
#   model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
  
#   # We start the training
#   epochs = 5
#   batch_size = 256
#   # We train it
#   r = model.fit(X_train_resized, Y_train,
#             batch_size=batch_size,
#             validation_data=(X_validate_resized, Y_validate),
#             epochs=epochs)

In [ ]:
classes = 27
def create_folder():
  import random
  import os
  from pathlib import Path

  !rm -r /content/train
  !rm -r /content/validate
  !rm -r /content/test

  !mkdir /content/train
  !mkdir /content/validate
  !mkdir /content/test

  for i in range(1,classes):
    os.mkdir("/content/train/Person"+str(i))
    os.mkdir("/content/validate/Person"+str(i))
    os.mkdir("/content/test/Person"+str(i))

In [ ]:
from pathlib import Path
# root = "/content/P1E_S1/"
def divide_data(root):
  for cam in os.listdir(root):
    for person in os.listdir(root+cam):
      images = os.listdir(root+cam+'/'+person+'/')
      random.shuffle(images)
      
      for i in range(round(len(images)*0.6)):
        Path(root+cam+'/'+person+'/'+images[i]).rename("/content/train/" + person+'/'+images[i])
      
      for i in range(round(len(images)*0.6),round(len(images)*0.9)):
        Path(root+cam+'/'+person+'/'+images[i]).rename("/content/validate/" + person+'/'+images[i])
      
      for i in range(round(len(images)*0.9),len(images)):
        Path(root+cam+'/'+person+'/'+images[i]).rename("/content/test/" + person+'/'+images[i])

In [ ]:
import cv2
import numpy as np
import os

x_train = []
y_train = []
x_validate  = []
y_validate  = []
x_test  = []
y_test  = []

#t=train/test/validate
def create_array(t):
  for person in os.listdir("/content/"+t+"/"):
    for img in os.listdir("/content/"+t+"/"+person):
      im_data = cv2.imread("/content/"+t+"/"+person+'/'+img,3)
      im_data = cv2.resize(im_data, (48,48))

      if t=="train":
        x_train.append(np.array(im_data))
        y_train.append(np.array([int(person[6:])-1]))
      if t=="test":
        x_test.append(np.array(im_data))
        y_test.append(np.array([int(person[6:])-1]))
      if t=="validate":
        x_validate.append(np.array(im_data))
        y_validate.append(np.array([int(person[6:])-1]))

In [ ]:
# We first load the necessary libraries, the dataset and reshape its dimensons to the minimum allowed by the VGG16 --> (48,48,3)
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from keras.engine import Model
from keras.layers import Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras.applications import VGG16, ResNet50
from keras.datasets import cifar10
from keras.utils import to_categorical
import numpy as np
input_shape = (48, 48, 3)

In [ ]:
!rm -r /content/abc

In [ ]:
from flask import Flask, render_template , request , jsonify, send_file
from flask_ngrok import run_with_ngrok
from zipfile import ZipFile
from PIL import Image
import datetime
import os , io , sys
import numpy as np 
import cv2
import base64
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

data = {"admin":"admin"}

@app.route('/add_person')
def add_person():
  return render_template('add.html')

@app.route('/add' , methods=['POST'])
def add():
  file = request.files.get('image')
  print(file)

  x = datetime.datetime.now()

  try:
    os.mkdir("/content/abc")
  except Exception as e:
    print(e)

  target_path = "/content/abc/" +str(x)+file.filename
  file.save(target_path)
  unzip_file(target_path, '/content/abc')
  zzz = file.filename
  folder_path = '/content/abc/' + zzz.split('.')[0]

  os.makedirs("/content/abc/temp/cam")
  os.system("mv "+folder_path + " /content/abc/temp/cam")

  print("Extraction complete")

  #create proper data
  global classes
  classes+=1
  create_folder()
  print("created folders-test,train& validate")

  #divide data as test,train and validate
  for i in ['/content/P1E_S1/','/content/P1E_S2/','/content/P1E_S3/','/content/abc/temp/']:
    divide_data(i)

  print("randomly divided dataset into train test and validate")
  #create image array
  create_array("train")
  create_array("test")
  create_array("validate")
  print("image to array converted")
  #-------------------------------------
  import numpy as np
  X_train=np.array(x_train)
  Y_train=np.array(y_train)

  X_validate=np.array(x_validate)
  Y_validate=np.array(y_validate)

  X_test=np.array(x_test)
  Y_test=np.array(y_test)

  Y_train = to_categorical(Y_train)
  Y_validate = to_categorical(Y_validate)
  #
  #-------------------------------------------
  # resize train set
  X_train_resized = []
  for img in X_train:
    X_train_resized.append(np.resize(img, input_shape) / 255)
    
  X_train_resized = np.array(X_train_resized)
  print(X_train_resized.shape)

  # resize test set
  X_validate_resized = []
  for img in X_validate:
    X_validate_resized.append(np.resize(img, input_shape) / 255)
    
  X_validate_resized = np.array(X_validate_resized)
  print(X_validate_resized.shape)


  print("Building New model")
  #add person
  # add_class()
  #-----------------
  from keras.models import Model
  import numpy as np

  global model
  model.summary()
  # save the original weights
  weights_bak = model.layers[-1].get_weights()
  nb_classes = model.layers[-1].output_shape[-1]
  # print(len(weights_bak[1]))
  model.layers.pop()
  # model.summary()
  # print("--------------------")
  new_layer = Dense(nb_classes + 1, activation='softmax')
  out = new_layer(model.layers[-2].output)
  inp = model.input
  model = Model(inp, out)
  # model.summary()
  weights_new = model.layers[-1].get_weights()

  # copy the original weights back
  weights_new[0][:, :-1] = weights_bak[0]
  weights_new[1][:-1] = weights_bak[1]

  # use the average weight to init the new class.
  weights_new[0][:, -1] = np.mean(weights_bak[0], axis=1)
  weights_new[1][-1] = np.mean(weights_bak[1])

  model.layers[-1].set_weights(weights_new)

  model.summary()
  #-----------------------
  from keras.optimizers import Adam
  model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
  
  # We start the training
  epochs = 3
  batch_size = 256
  # We train it
  r = model.fit(X_train_resized, Y_train,
            batch_size=batch_size,
            validation_data=(X_validate_resized, Y_validate),
            epochs=epochs)

  return "Added new person!"


@app.route('/maskImage' , methods=['POST'])
def mask_image():
  
  file = request.files['image'].read() ## byte file
  npimg = np.fromstring(file, np.uint8)

  img = cv2.imdecode(npimg,cv2.IMREAD_COLOR)

  cv2.imwrite("abc.jpg",img)
  usrObj = User()
  usrObj.upload('/content/abc.jpg')
  match_id = usrObj.get_results()

  #no face found
  if match_id == "0":
    return render_template('index.jinja2',flag=True)

  # match_id = find_match("/content/abc.jpg")
  
  with open("/content/labels/"+str(match_id)+".jpg", "rb") as img_file:
    my_string = base64.b64encode(img_file.read())

  return jsonify({'status':str(my_string)})


@app.route('/home')
def home():
  return render_template('home.html')

@app.route('/index')
def index():
  return render_template('index.html')

@app.route('/match')
def match():
  return render_template('index.jinja2')
  # return render_template('home.html')

@app.route('/login')
def login():
  return render_template('login.html')

@app.route('/logout')
def logout():
  return render_template('index.html')

@app.route('/lo', methods=["POST"])
def lo():
  username = request.form.get('Uname')
  pwd = request.form.get('Pass')
  print(username,pwd)
  if username=="admin" and pwd==data["admin"]:
    # return "admin"
    return render_template('admin.html')

  if username in data.keys():
    if pwd==data[username]:
      # return "user"
      return render_template('home.html')

  return render_template('login.html',flag=True)

@app.route('/sign',methods=["POST"])
def sign():
  username = request.form['Uname']
  pwd = request.form['Pass']

  if username in data.keys():
    return render_template('signup.html',flag=True)
  
  data[username]=pwd
  return render_template('login.html')

@app.route('/signup')
def signup():
  return render_template('signup.html')

@app.route("/static_ip/")
def static_ip():
  return send_file("/content/ip.png")

@app.route("/static_op/")
def static_op():
  return send_file("/content/op.png")

@app.route("/static_load/")
def static_load():
  return send_file("/content/load.png")



if __name__ == '__main__':
	app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://a2efad43cc9a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [06/Apr/2021 08:00:30] "GET /match HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 08:00:30] "GET /static/index.css HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 08:00:30] "GET /static/index.js HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 08:00:31] "GET /static_ip HTTP/1.1" 308 -
127.0.0.1 - - [06/Apr/2021 08:00:31] "GET /static_op HTTP/1.1" 308 -
127.0.0.1 - - [06/Apr/2021 08:00:32] "GET /static_ip/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 08:00:32] "GET /static_op/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Apr/2021 08:00:33] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [06/Apr/2021 08:00:37] "GET /static_load HTTP/1.1" 308 -
127.0.0.1 - - [06/Apr/2021 08:00:38] "GET /static_load/ HTTP/1.1" 200 -
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:147: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
127.0.0.1 - - [06/Apr/2021 08:00:41] "POST /maskImage HTTP/1.1" 200 -


14


...

127.0.0.1 - - [06/Apr/2021 08:09:19] "GET /lo HTTP/1.1" 405 -
127.0.0.1 - - [06/Apr/2021 08:09:31] "GET /add_person HTTP/1.1" 200 -


<FileStorage: 'Person27.zip' ('application/x-zip-compressed')>
Extraction complete
rm: cannot remove '/content/train': No such file or directory
rm: cannot remove '/content/validate': No such file or directory
rm: cannot remove '/content/test': No such file or directory
created folders-test,train& validate
randomly divided dataset into train test and validate
image to array converted
(5806, 48, 48, 3)
(2913, 48, 48, 3)
Building New model
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (Ma

[2021-04-06 08:14:47,419] ERROR in app: Exception on /add [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.7/dist-packages/flask_cors/extension.py", line 165, in wrapped_function
    return cors_after_request(app.make_response(f(*args, **kwargs)))
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.7/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.7/dist-packages/flask/app.py", line 1936, in dispatch_request
 

<FileStorage: 'Person27.zip' ('application/x-zip-compressed')>
[Errno 17] File exists: '/content/abc'
23/23 [==============================] - 112s 5s/step - loss: 0.0580 - accuracy: 0.9900 - val_loss: 0.0558 - val_accuracy: 0.9870


127.0.0.1 - - [06/Apr/2021 08:15:28] "POST /add HTTP/1.1" 200 -


# extra

In [ ]:
!wget https://github.com/YuvalNirkin/face_segmentation/releases/download/1.0/face_seg_fcn8s.zip

In [ ]:
!wget https://github.com/YuvalNirkin/face_segmentation/releases/download/1.1/face_seg_fcn8s_300_no_aug.zip

In [ ]:
!unzip /content/face_seg_fcn8s.zip
!unzip /content/face_seg_fcn8s_300_no_aug.zip

In [ ]:
!git clone https://github.com/YuvalNirkin/face_segmentation.git

In [ ]:
%cd /content/face_segmentation/interfaces/python
!python face_seg.py

In [ ]:
# !sudo apt install caffe-cpu
!apt-get update
!sudo apt install caffe-cuda

In [ ]:
# !mkdir /content/face_segmentation
!git clone https://github.com/criminalking/face_segmentation.git

In [ ]:
%cd /content/face_segmentation

In [ ]:
!pip install -r requirements.txt

In [ ]:
pip install caffe

ERROR: Could not find a version that satisfies the requirement caffe (from versions: none)
ERROR: No matching distribution found for caffe


In [ ]:
!python2 face_segment_part.py

In [ ]:
# !rm -r /content/face_segmentation/

In [ ]:
!zip -r static.zip /content/static/
!zip -r templates.zip /content/templates

  adding: content/static/ (stored 0%)
  adding: content/static/index.css (deflated 49%)
  adding: content/static/index.js (deflated 63%)
  adding: content/static/login.css (deflated 66%)
  adding: content/static/.ipynb_checkpoints/ (stored 0%)
  adding: content/templates/ (stored 0%)
  adding: content/templates/login.html (deflated 58%)
  adding: content/templates/add.html (deflated 42%)
  adding: content/templates/index.jinja2 (deflated 52%)
  adding: content/templates/home.html (deflated 37%)
  adding: content/templates/signup.html (deflated 58%)
  adding: content/templates/admin.html (deflated 39%)
  adding: content/templates/index.html (deflated 38%)
  adding: content/templates/.ipynb_checkpoints/ (stored 0%)


In [ ]:
# %cd /content/abc/
# !unzip static.zip
# !rm -r /contet
%cd /content/

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-94-4e8a362026c0>", line 4, in <module>
    get_ipython().magic('cd /content/')
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2160, in magic
    return self.run_line_magic(magic_name, magic_arg_s)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2081, in run_line_magic
    result = fn(*args,**kwargs)
  File "<decorator-gen-84>", line 2, in cd
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magic.py", line 188, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/magics/osm.py", line 288, in cd
    oldcwd = py3compat.getcwd()
FileNotFoundError: [Errno 2] No such file or directory

During handling of the above exception, anothe

FileNotFoundError: ignored